# Data Wrangling - Assignment 1

## 0. Setup

In [ ]:
%pip install -r requirements.txt

In [15]:
from pathlib import Path
from shutil import rmtree
from typing import Dict, List, Tuple
import sqlite3

import numpy as np
import pandas as pd
import requests

from bs4 import BeautifulSoup

%matplotlib inline

## 1. Fetching Data and Preprocessing

If `FETCH_DATA` is set to True, the raw data from previous executions will be deleted. All data is then fetched again from these data sources:


| Data                      | Source                         | URL                                                                                      | Datatype  |
|:---                       |:---                            |:---                                                                                      | :---      |
| Basic Data on Communes    | Bundesamt für Statistik        | https://dam-api.bfs.admin.ch/hub/api/dam/assets/15864450/master                          | XLSX File |
| Coat of Arms of Communes  | Staatsarchiv Kanton Luzern     | https://staatsarchiv.lu.ch/kantonsgeschichte/gemeinden/Gemeindewappen/*\[commune\]*      | JPG Files |
| Points of Interest        | Geoportal Kanton Luzern        | https://public.geo.lu.ch/ogd/rest/services/managed/DIGOPPOI_COL_V1_MP/MapServer/1/quer   | JSON File |

In [43]:
FETCH_DATA = False
RAW_DATA_PATH = Path().cwd() / 'raw_data'
COMMUNES_PATH = RAW_DATA_PATH /  'communes.xlsx'
COAT_OF_ARMS_PATH = RAW_DATA_PATH / 'coat_of_arms'
POI_PATH = RAW_DATA_PATH / 'poi'

In [3]:
def delete_raw_data(raw_data_dir: Path, subdirs: List[Path]) -> None:
    '''
    Deletes all raw data if the raw data directory exists.

    Parameters
    ----------
    raw_data_dir : Path
        The path of the directory that contains the raw data.
    subdirs : list[Path]
        A list of subdirs as an absolute path to create inside the raw data directory.
    '''
    if raw_data_dir.exists():
        rmtree(raw_data_dir)
    
    raw_data_dir.mkdir()

    for dir in subdirs:
        dir.mkdir()

In [4]:
if FETCH_DATA:
    delete_raw_data(RAW_DATA_PATH, [COAT_OF_ARMS_PATH])

### 1.1. Basic Data: XLSX File for Data on all Comunes

#### 1.1.1. Fetch XLSX File

In [5]:
def download_binary_file(url: str, destination_dir: Path) -> None:
    '''
    Downloads a file from a given URL via a HTTP GET request.

    Parameters
    ----------
    url : str
        The URL of the file.
    destination_dir : Path
        The path of directory in which the file should be stored.
    Returns
    -------
    None
    '''
    request = requests.get(url)
    request.raise_for_status()

    with open(destination_dir, "wb") as f:
        f.write(request.content)

In [6]:
if FETCH_DATA:
    bfs_url = 'https://dam-api.bfs.admin.ch/hub/api/dam/assets/15864450/master'
    download_binary_file(bfs_url, destination_dir=COMMUNES_PATH)

#### 1.1.2. Preprocess XLSX File

In [7]:
def preprocess_communes_df(raw_communes: pd.DataFrame, min_bfs_nr: int, max_bfs_nr: int) -> Tuple[pd.DataFrame, Dict[int, str]]:
    '''
    Applies the preprocessing steps to the raw communes DataFrame.

    Parameters
    ----------
    raw_communes : DataFrame
        The raw DataFrame after loading from an excel file.
    min_bfs_nr : int
        The BFS number
        (`see Wikipedia <https://en.wikipedia.org/wiki/Community_Identification_Number#Switzerland>`_)
        of the first commune to include in the preprocssed DataFrame.
    max_bfs_nr : int
        The BFS number of the last commune to include in the preprocessed
        DataFrame (The commune with this number will be included in the
        preprocessed DataFrame).
    
    Returns
    -------
    communes : pd.DataFrame
        The complete preprocessed DataFrame.
    communes_names : dict
        A DataFrame that contains only the names of the communes in the
        the range of BFS numbers passed to this function.
    '''
    
    # Improve column names by removing:
    # - '-' characters to
    # - unecessary references like 1)
    raw_communes.rename(
        columns = {
            raw_communes.columns[3]: raw_communes.columns[3].replace('-', ''),
            raw_communes.columns[14]: raw_communes.columns[14][:-3],
            raw_communes.columns[32]: raw_communes.columns[32][:-3],
        },
        inplace=True
    )

    raw_communes.drop(labels=np.nan, axis=0, inplace=True)
    raw_communes.set_index(raw_communes.index.astype('int'), inplace=True)

    # select communes from that have BFS numbers in the given range
    communes = raw_communes.loc[min_bfs_nr:max_bfs_nr]

    # Remove the LU suffix present for some communes
    communes.loc[:, 'Gemeindename'] = communes["Gemeindename"].str.replace(
        r'(?P<name>\w+) \(LU\)',
        lambda m: m.group('name'),
        regex=True,
    )

    communes_names = communes.loc[:, 'Gemeindename'].sort_values().to_dict()
    return communes, communes_names

In [8]:
communes = pd.read_excel(
    COMMUNES_PATH,
    skiprows=[0, 1, 2, 3, 4, 6, 7],
    skipfooter=16,
    index_col=0,
)

communes_lu, communes_lu_names = preprocess_communes_df(
    raw_communes=communes,
    min_bfs_nr=1001,
    max_bfs_nr=1151,
)

### 1.2. Additional Information: Gemeindewappen 

#### 1.2.1. Fetch JPG Files

In [9]:
def download_coat_of_arms(source_url: str, destination_dir: Path, commune_names: Dict[int, str], logging=False) -> None:
    '''
    Downloads images of all coats of arms from Staatsarchiv Luzern.

    Parameters
    ----------
    source_url : str
        The URL to download the images from.
    destination_dir : Path
        The Path of the directory where the coats of arms should be stored.
    commune_names : dict
        A dict that matches all BFS numbers of the communes to their names.
    logging : bool
        Wheter to print logging outputs or not. The Default is False,
        meaning no logging statements will be printed.
    
    Returns
    -------
    None
    '''
    # fetch images of coat of arms for all communes
    for cid, commune in commune_names.items():
        # handle concatenated and multi-word commune names
        if '-' in commune:
            commune = commune.split('-')[0]
        elif ' ' in commune:
            commune = commune.split(' ')[0]
        # The coat of arms of 'Willisau' is stored under 'Willisau-Stadt'
        # in https://staatsarchiv.lu.ch
        elif commune == 'Willisau':
            commune = 'Willisau-Stadt'

        current_url = source_url + commune
        if logging:
            print(f'Fetching {current_url}')
        request = requests.get(current_url)
        request.raise_for_status()

        soup = BeautifulSoup(request.text, 'html.parser')
        img = soup.find(id='maincontent_1_imgImage')
        img_url = 'https://staatsarchiv.lu.ch' + img.get('src')

        img_request = requests.get(img_url)
        img_request.raise_for_status()

        current_img_path = destination_dir / f'{cid}.jpg'
        with open(current_img_path, "wb") as f:
            f.write(img_request.content)

In [10]:
if FETCH_DATA:
    sa_lu_url = 'https://staatsarchiv.lu.ch/kantonsgeschichte/gemeinden/Gemeindewappen/'
    download_coat_of_arms(
        sa_lu_url, COAT_OF_ARMS_PATH,
        communes_lu_names, logging=True
    )

### 1.3. Additional Information: Points of Interest



In [47]:
def download_points_of_interest(soruce_url: str, destination_dir: Path) -> None:
    '''
    Downloads a JSON file of all points of interest (POI) from a MapServer using
    a HTTP POST request.

    Parameters
    ----------
    source_url : str
        The URL to download the points of interest from.
    destination_dir : Path
        The Path of the directory where the downloaded data should be stored.
    
    Returns
    -------
    None
    '''

    # API options in post body
    payload = {
        'f': 'pjson', # set output to json
        'outFields': 'BFS_NR,POI_NAME,INFO_URL', # select relevant fields
        'returnGeometry': 'true', # get the coordinates of the POI
        'units': 'esriSRUnit_Meter' #  set the untis to meters
    }

    # Fetch data from ESRI MapServer
    request = requests.post(soruce_url, data=payload)
    request.raise_for_status()

    with open(destination_dir / 'poi.json', 'wt', encoding='utf-8') as f:
        f.write(request.text)

In [49]:
if FETCH_DATA:
    geo_lu_url = 'https://public.geo.lu.ch/ogd/rest/services/managed/DIGOPPOI_COL_V1_MP/MapServer/1/query'
    download_points_of_interest(
        soruce_url=geo_lu_url,
        destination_dir=POI_PATH
    )

## 2. Feature Extraction

Extract the relevant features from all downloaded data sources. 

### 2.1. Extract Relevant Feaures from communes file

In [11]:
base_data = communes_lu.iloc[:, [0, 1, 3, 5, 6, 7, 14, 15, 17, 19, 29, 30]]
base_data

,Gemeindename,Einwohner,Bevölkerungsdichte pro km²,0-19 Jahre,20-64 Jahre,65 Jahre und mehr,Gesamtfläche in km²,Siedlungsfläche in %,Landwirtschafts-fläche in %,Wald und Gehölze in %,Leerwohnungs-ziffer,Neu gebaute Wohnungen pro 1000 Einwohner
Gemeindecode,,,,,,,,,,,,
1001,Doppleschwand,789,113.525180,26.742712,58.681876,14.575412,6.95,5.755396,55.683453,36.115108,1.704545,1.272265
1002,Entlebuch,3280,57.644991,21.798780,59.664634,18.536585,56.90,3.686151,49.552396,42.829559,1.092896,8.446456
1004,Flühli,1929,17.833041,21.876620,60.290306,17.833074,108.17,2.497918,44.527708,38.236655,1.052150,5.112474
1005,Hasle,1736,43.066237,22.695853,59.907834,17.396313,40.31,3.248202,49.094967,39.573518,2.241594,17.694064
1007,Romoos,659,17.625033,22.003035,59.787557,18.209408,37.39,2.088353,35.796519,60.803213,0.874636,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
1145,Ufhusen,888,72.727273,21.959459,60.585586,17.454955,12.21,5.496308,73.256768,20.918786,1.250000,15.503876
1146,Wauwil,2261,763.851351,20.477665,64.838567,14.683768,2.96,21.088435,68.707483,10.204082,4.360465,11.483693
1147,Wikon,1520,183.574879,22.828947,59.210526,17.960526,8.28,9.650181,38.359469,51.869723,0.722543,6.557377


### 2.2. Extract Features from coats of arms

Color Red in HSV Colormodel:

$H = 0..10$, $345..360$

$S \ge 50$

$V \ge 50$

### 2.3. Merge all DataFrames

In [12]:
lu_data = base_data

## 3. Create Ranking

In [14]:
display(lu_data.loc[:, ['Gemeindename', 'Bevölkerungsdichte pro km²']].sort_values('Bevölkerungsdichte pro km²'))
display(lu_data.loc[:, ['Gemeindename', 'Wald und Gehölze in %']].sort_values('Wald und Gehölze in %', ascending=False))
display(lu_data.loc[:, ['Gemeindename', 'Neu gebaute Wohnungen pro 1000 Einwohner']].sort_values('Neu gebaute Wohnungen pro 1000 Einwohner', ascending=False))


,Gemeindename,Bevölkerungsdichte pro km²
Gemeindecode,,
1007,Romoos,17.625033
1004,Flühli,17.833041
1135,Luthern,32.786017
1010,Escholzmatt-Marbach,40.892857
1005,Hasle,43.066237
...,...,...
1052,Buchrain,1325.208333
1054,Ebikon,1430.681818
1024,Emmen,1523.956799


,Gemeindename,Wald und Gehölze in %
Gemeindecode,,
1007,Romoos,60.803213
1066,Schwarzenberg,54.533877
1068,Vitznau,54.199328
1147,Wikon,51.869723
1059,Kriens,50.458716
...,...,...
1023,Ballwil,11.771429
1088,Hildisrieden,11.731044
1031,Hochdorf,10.460251


,Gemeindename,Neu gebaute Wohnungen pro 1000 Einwohner
Gemeindecode,,
1055,Gisikon,30.803907
1125,Dagmersellen,25.144347
1052,Buchrain,24.689339
1064,Meierskappel,24.105186
1100,Schlierbach,23.917995
...,...,...
1056,Greppen,0.000000
1053,Dierikon,0.000000
1039,Römerswil,0.000000
